In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random
import string

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins with sk-proj")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = (
    "You are a helpful assistant for an e-commerce company called SparksAI.\n\n"
    "When the user wants to buy a product, follow these steps:\n"
    "1. Ask for the name of the product.\n"
    "2. Ask for the quantity they want.\n"
    "3. Call the function 'check_product_availability' with the name of the product the user wants to buy.\n"
    "   - If it returns an empty list, say: 'Product not available'.\n"
    "   - If it returns products, list them EXACTLY, in a numbered list, showing name, quantity and price.\n"
    "4. Ask the customer if they want something else. If yes then repeat steps 1 to 3.\n"
    "5. Wait for the user to confirm the selection.\n"
    "6. Then ask for customer's first name, last name, and email.\n"
    "7. Ask for the complete delivery address in the following format.\n"
    "   - Address Line 1 \n"
    "   - Suite No., Apt No., Building No. (optional)\n"
    "   - Zip Code\n"
    "   - Phone number\n"
    "6. Finally call 'order_product' to confirm and show the user the real order details with the generated order number and all the details of the product that they ordered along with the total amount that they have to pay.\n"
    "- Always call 'check_product_availability' if user mentions a new product.\n"
    "- Do not invent products. Use what the function calls return.\n"
    "- If a product is not available, say 'Product not found'.\n"
    "If you don't know something, say so.\n"
    "Keep answers precise and professional.\n"
)

In [ ]:
available_products = {
    "fevicol": [
        {
            "name": "Fevicol PL 333",
            "quantity": "5 litres",
            "price": "$20"
        },
        {
            "name": "Fevicol PL 111",
            "quantity": "5 litres",
            "price": "$25"
        },
        {
            "name": "Fevicol SR 505",
            "quantity": "5 litres",
            "price": "$30"
        },
    ],
    "loctite": [
        {
            "name": "Loctite 401",
            "quantity": "50 ml",
            "price": "$25"
        },
        {
            "name": "Loctite 243",
            "quantity": "250 ml",
            "price": "$40"
        },
        {
            "name": "Loctite 638",
            "quantity": "50 ml",
            "price": "$30"
        }
    ],
    "3m": [
        {
            "name": "3M Scotch-Weld 2216",
            "quantity": "200 ml",
            "price": "$50"
        },
        {
            "name": "3M Super 77",
            "quantity": "500 ml",
            "price": "$35"
        },
        {
            "name": "3M Fastbond 30",
            "quantity": "1 gallon",
            "price": "$80"
        }
    ],
    "bosch": [
        {
            "name": "Bosch PU Adhesive",
            "quantity": "1 litre",
            "price": "$28"
        },
        {
            "name": "Bosch Epoxy Resin",
            "quantity": "500 ml",
            "price": "$18"
        },
        {
            "name": "Bosch High-Strength Glue",
            "quantity": "750 ml",
            "price": "$22"
        }
    ],
    "araldite": [
        {
            "name": "Araldite Standard Epoxy",
            "quantity": "500 gm",
            "price": "$20"
        },
        {
            "name": "Araldite Rapid Adhesive",
            "quantity": "1 litre",
            "price": "$35"
        },
        {
            "name": "Araldite Super Strength",
            "quantity": "250 ml",
            "price": "$15"
        }
    ]
}

In [ ]:
def generate_order_number(prefix="SPK", length=8):
    random_part = ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))
    return f"{prefix}-{random_part}"

def check_product_availability(product_name: str):
    print(f"[TOOL] check_product_availability({product_name})")
    product = product_name.lower()
    return available_products.get(product, [])

def order_product(order_details):
    print(f"Order confirmed! Your order number is {order_details['order_number']}.")
    print("Order details:")
    for item in order_details["products"]:
        print(f"- {item['name']} x{item['quantity']} - ${item['price']}")


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_product_availability",
            "description": "Checks if a product is available in inventory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {"type": "string", "description": "Name of the product to check."}
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_product",
            "description": "Places an order for a product.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_number": {"type": "string", "description": "Unique order number."},
                    "customer": {
                        "type": "object",
                        "properties": {
                            "first_name": {"type": "string"},
                            "last_name": {"type": "string"},
                            "email": {"type": "string"}
                        },
                        "required": ["first_name", "last_name", "email"]
                    },
                    "delivery_address": {
                        "type": "object",
                        "properties": {
                            "address_line1": {"type": "string"},
                            "address_line2": {"type": "string"},
                            "zip_code": {"type": "string"},
                            "phone_number": {"type": "string"}
                        },
                        "required": ["address_line1", "zip_code", "phone_number"]
                    },
                    "products": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": {"type": "string"},
                                "quantity": {"type": "integer"},
                                "price": {"type": "number"}
                            },
                            "required": ["name", "quantity", "price"]
                        }
                    }
                },
                "required": ["order_number", "customer", "delivery_address", "products"]
            }
        }
    }
]

In [ ]:
def handle_tool_call(tool_call):
    """
    Handles a single function call requested by the LLM.
    """
    fn_name = tool_call.function.name  # Use dot notation
    args = json.loads(tool_call.function.arguments)  # Convert JSON string to dictionary

    if fn_name == "check_product_availability":
        product_name = args.get("product_name")
        available_products = check_product_availability(product_name)  # Ensure this function exists
        response_content = {"product_name": product_name, "availability": available_products}

    elif fn_name == "order_product":
        customer = args.get("customer")
        address = args.get("delivery_address")
        products = args.get("products")
        order_number = generate_order_number()

        order_details = {
            "order_number": order_number,
            "customer": customer,
            "delivery_address": address,
            "products": products
        }

        order_product(order_details)  # Ensure this function exists
        response_content = {"order_number": order_number, "order_details": order_details}

    elif fn_name == "generate_order_number":
        prefix = args.get("prefix", "SPK")
        length = args.get("length", 8)
        order_number = generate_order_number(prefix, length)
        response_content = {"order_number": order_number}

    else:
        response_content = {"error": f"Unknown function: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,  # Use dot notation
    }, args

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools  # Ensure this is a correctly defined list of tools
        )

        # Check if OpenAI wants to call a tool
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message  # This is a ChatCompletionMessage object

            print("[DEBUG] msg.tool_calls:", msg.tool_calls)

            if isinstance(msg.tool_calls, list) and msg.tool_calls:
                print("[DEBUG] First tool call:", msg.tool_calls[0])
            else:
                print("[ERROR] No valid tool call found")

            tool_call = msg.tool_calls[0]  # Extract the tool call object

            print(f"[INFO] Tool call requested: {tool_call}")

            # Extract function name and arguments safely
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)  # Convert JSON string to dictionary

            # Handle the tool call
            tool_response, tool_args = handle_tool_call(tool_call)

            print(f"[INFO] Tool response: {tool_response}")

            messages.append({
                "role": "assistant",
                "content": None,
                "tool_calls": [{
                    "id": tool_call.id,
                    "type": "function",  # Required field
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": json.dumps(fn_args)  # Ensure this is a valid JSON string
                    }
                }]
            })

            messages.append({
                "role": "tool",
                "content": json.dumps(tool_response),
                "tool_call_id": tool_call.id
            })

            # Re-send updated conversation to get final or next step
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                tools=tools
            )

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()